# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [130]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json 

# Import API key
from api_keys import geoapify_key

In [131]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachnyy,66.4167,112.4000,235.62,73,98,1.14,RU,1675818096
1,1,jamestown,42.0970,-79.2353,281.92,75,100,6.17,US,1675817815
2,2,nanortalik,60.1432,-45.2371,257.85,72,8,8.65,GL,1675818107
3,3,tukrah,32.5341,20.5791,284.52,65,99,4.81,LY,1675818108
4,4,half moon bay,37.4636,-122.4286,289.40,67,0,6.69,US,1675818108


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [132]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    scale = 1,
    size ="Humidity",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [136]:
# Narrow down cities that fit criteria and drop any results with null values
    
# Drop any rows with null values
city_data_clean = city_data_df.dropna()

# Itterate thorugh city temperatures 
city = [] 

for index, row in city_data_clean.iterrows():
    if row ["Max Temp"] <= 300:
        if row ["Cloudiness"] <25:
            if row ["Wind Speed"] <=5:
                if row["Humidity"] <=60:
                    city.append(row)
                
city_ideal_weather_df = pd.DataFrame(city)
   
# Display sample data
city_ideal_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,ushuaia,-54.8000,-68.3000,284.96,50,0,1.54,AR,1675818110
28,28,busselton,-33.6500,115.3333,295.57,57,0,3.71,AU,1675818118
36,36,chepareria,1.3077,35.2039,288.71,57,8,2.29,KE,1675818143
40,40,new norfolk,-42.7826,147.0587,289.66,36,4,0.94,AU,1675818157
80,80,gariaband,20.6350,82.0614,287.42,44,0,0.78,IN,1675818224
84,84,nara,34.6851,135.8049,282.02,56,8,3.41,JP,1675818231
88,88,huntington beach,33.6603,-117.9992,294.36,43,20,4.63,US,1675818235
117,117,port arthur,38.8000,121.2667,271.27,45,0,4.48,CN,1675818275
136,136,mount gambier,-37.8333,140.7667,294.41,46,5,4.54,AU,1675818112
139,139,kwinana,-32.2500,115.7667,299.16,56,0,0.89,AU,1675818293


### Step 3: Create a new DataFrame called `hotel_df`.

In [137]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,ushuaia,AR,-54.8000,-68.3000,50,
28,busselton,AU,-33.6500,115.3333,57,
36,chepareria,KE,1.3077,35.2039,57,
40,new norfolk,AU,-42.7826,147.0587,36,
80,gariaband,IN,20.6350,82.0614,44,
84,nara,JP,34.6851,135.8049,56,
88,huntington beach,US,33.6603,-117.9992,43,
117,port arthur,CN,38.8000,121.2667,45,
136,mount gambier,AU,-37.8333,140.7667,46,
139,kwinana,AU,-32.2500,115.7667,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [139]:
# Set parameters to search for a hotel
radius = 100000

params = {
     "catergories": "accomidation/hotel",
     "apikey":geoapify_key
 }
    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params['filters'] = filters
    params['bias'] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    print(params)
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'catergories': 'accomidation/hotel', 'apikey': '5d8cd70e27c340d3874a692b65582290', 'filters': 'circle:-68.3,-54.8,100000', 'bias': 'proximity:-68.3,-54.8'}
ushuaia - nearest hotel: No hotel found
{'catergories': 'accomidation/hotel', 'apikey': '5d8cd70e27c340d3874a692b65582290', 'filters': 'circle:115.3333,-33.65,100000', 'bias': 'proximity:115.3333,-33.65'}
busselton - nearest hotel: No hotel found
{'catergories': 'accomidation/hotel', 'apikey': '5d8cd70e27c340d3874a692b65582290', 'filters': 'circle:35.2039,1.3077,100000', 'bias': 'proximity:35.2039,1.3077'}
chepareria - nearest hotel: No hotel found
{'catergories': 'accomidation/hotel', 'apikey': '5d8cd70e27c340d3874a692b65582290', 'filters': 'circle:147.0587,-42.7826,100000', 'bias': 'proximity:147.0587,-42.7826'}
new norfolk - nearest hotel: No hotel found
{'catergories': 'accomidation/hotel', 'apikey': '5d8cd70e27c340d3874a692b65582290', 'filters': 'circle:82.0614,20.635,100000', 'bias': 'proximity:82.0614,2

,City,Country,Lat,Lng,Humidity,Hotel Name
10,ushuaia,AR,-54.8000,-68.3000,50,No hotel found
28,busselton,AU,-33.6500,115.3333,57,No hotel found
36,chepareria,KE,1.3077,35.2039,57,No hotel found
40,new norfolk,AU,-42.7826,147.0587,36,No hotel found
80,gariaband,IN,20.6350,82.0614,44,No hotel found
84,nara,JP,34.6851,135.8049,56,No hotel found
88,huntington beach,US,33.6603,-117.9992,43,No hotel found
117,port arthur,CN,38.8000,121.2667,45,No hotel found
136,mount gambier,AU,-37.8333,140.7667,46,No hotel found
139,kwinana,AU,-32.2500,115.7667,56,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [102]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 600,
    frame_length = 800,
    color = "City"
    )

# Display the map
# YOUR CODE HERE